# LangChain Use Cases

This document dives stragith into implementing [LangChain use case](https://docs.langchain.com/docs/category/use-cases) using a step-by-step approach, and hopefully inspire you to build.

## Use Case - Summarization

Langchain summarization is one of the most common usage of LangChain and LLMs. It can summarize any amount of text or documentations, including the ones that exceeds the token limit (currently set at `4096 tokens` for `gpt-35-turbo`), which roughly equates to `3000 words`. 

The technique used is similar to the concept of `sliding window`, which a fixed size window, usually under the max context window limit, is used to chunk the the long document into smaller pieces, then summarize the content recursively to produce the final summary using mapreduce.

You can use it to summarize not only text, books, documents, audios, social media threads, and etc. Some use cases including: _articles_, _research pagers_, _legal and financial documents_, _transcripts_, _chat history_, _customer interactions_, _product reviews_, _podcasts_, _twitter threads_ and more.

In [4]:
import dotenv
import os
import pdfplumber
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

dotenv.load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(temperature = 0, openai_api_key = openai_api_key)

Now, let's load in `The Tragedy of Hamlet`

In [5]:
with pdfplumber.open("hamlet.pdf") as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
        
print(text[:300])

The Tragedy of Hamlet, Prince of
Denmark
ASCIItextplacedinthepublicdomainbyMobyLexicalTools,1992.SGMLmarkupbyJonBosak,
1992-1994.XMLversionbyJonBosak,1996-1999.SimplifiedXMLversionbyMaxFroumentin,2001.The
XMLmarkupinthisversionisCopyright©1999JonBosak.Thisworkmayfreelybedistributedoncondition
thatit


By outputting the total number of tokens of the book with [get_num_tokens](https://python.langchain.com/en/latest/reference/modules/llms.html#langchain.llms.OpenAI.get_num_tokens), you can clearly see it exceeds the context window limit.

In [7]:
n_tokens = llm.get_num_tokens(text)

print(f"total number of tokens: {n_tokens}")

total number of tokens: 53929


The process starts by `splitting` the text into multiple parts by `chunk size` using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html), with a `coverlap` of each adjacent chunk of text.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=2000, chunk_overlap=200, length_function = len)
docs = text_splitter.create_documents([text])

print(f"number of docs: {len(docs)}")

number of docs: 100


Now the docs are ready, we can then load up a `chain` to do produce the sum of sums.

In [12]:
chain = load_summarize_chain(llm = llm, chain_type = 'map_reduce')
result = chain.run(docs)

print(result)

 In The Tragedy of Hamlet, Prince of Denmark, Hamlet is on a quest for revenge against his uncle, Claudius, who has usurped the throne. After encountering a ghostly figure, Hamlet learns that his father was murdered by his uncle and is determined to avenge his death. King Claudius and Lord Polonius devise a plan to spy on Hamlet and Ophelia, and Hamlet kills Polonius. Laertes returns from France and challenges Hamlet to a fencing match, during which Laertes wounds Hamlet with a poisoned sword. Hamlet stabs King Claudius with a poisoned sword and forces him to drink a poisoned potion. Prince Fortinbras arrives and orders that the bodies be placed on a stage for all to see, and then claims his rights of memory in the kingdom.


When instanciating the `chain`, you can also add `Verbose = True` to see the steps LangChain took before producing the final summary, i.e. generating a summary per each chunk of text, and produce the final summary by aggregating the 100 summaries with map reduce.

This is a powerful technique to overcome the token limitation. Even though more powerful models will be released in the future to support more tokens, such as Claude now supports [100K tokens](https://www.anthropic.com/index/100k-context-windows) as of 11 May 2023, biggest of the its kind. There is not guaranteed better performance, accuracy or cost-effectiveness.

Moreover, you can [parallelise the calls](https://github.com/hwchase17/langchain/issues/1073) to super charge summarization using `batch_size` when instanciating LLMs.